In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [113]:
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import xgboost as xgb
import random
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (30,10)
import numpy as np 
import pandas as pd 
from xgboost import plot_importance
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 

In [81]:
df_train=pd.read_csv("drive/MyDrive/Datasets-Collab/competitive-data-science-predict-future-sales/sales_train.csv")
df_test=pd.read_csv("drive/MyDrive/Datasets-Collab/competitive-data-science-predict-future-sales/test.csv")
shop=pd.read_csv("drive/MyDrive/Datasets-Collab/competitive-data-science-predict-future-sales/shops.csv")
items=pd.read_csv("drive/MyDrive/Datasets-Collab/competitive-data-science-predict-future-sales/items.csv")
item_categories=pd.read_csv("drive/MyDrive/Datasets-Collab/competitive-data-science-predict-future-sales/item_categories.csv")

In [82]:
df_train["date"]=pd.to_datetime(df_train["date"],format='%d.%m.%Y')
df_train['day']=df_train['date'].dt.day
df_train['month']=df_train['date'].dt.month
df_train['year']=df_train['date'].dt.year
df_train=df_train.sort_values(by='date')
df_test['month']=11
df_test['year']=2015


In [83]:
df_train['item_cnt_day']=abs(df_train['item_cnt_day'])
not_in_train=set(list(np.sort(df_test['item_id'].unique()))) - set(list(np.sort(df_train['item_id'].unique())))
df_train.drop(df_train.loc[df_train.item_id==6066].index,axis=0,inplace=True)

##Concatenating train and test set

In [84]:
df_train=pd.merge(df_train,items[['item_id','item_category_id']],on='item_id')
df_test=pd.merge(df_test,items[['item_id','item_category_id']],on='item_id')

In [85]:
df_train=df_train.groupby(['date_block_num','year','month','shop_id','item_id','item_category_id']).agg({'item_price': 'mean', 
                                                                    'item_cnt_day': 'mean'
                                                                   }).reset_index()

In [86]:
merged=pd.concat([df_train,df_test])

In [87]:
merged['High_sale_month']=0
merged.loc[(merged['month']>=9),'High_sale_month']=1

## Splitting Sequentially

In [88]:
merged.drop(['item_price'],inplace=True,axis=1)


In [89]:
merged.isnull().any()

date_block_num       True
year                False
month               False
shop_id             False
item_id             False
item_category_id    False
item_cnt_day         True
ID                   True
High_sale_month     False
dtype: bool

In [90]:
final_df=merged

In [91]:
df_train.shape

(1609123, 8)

In [92]:
df_train_final=final_df.iloc[:1609123,:]
df_test_final=final_df.iloc[1609123:,:]

In [93]:
df_train_final.drop(['ID'],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [94]:
df_train_final.isnull().any()

date_block_num      False
year                False
month               False
shop_id             False
item_id             False
item_category_id    False
item_cnt_day        False
High_sale_month     False
dtype: bool

In [95]:
X_train=df_train_final.loc[df_train_final['date_block_num']<32].drop(['item_cnt_day'],axis=1)
Y_train=df_train_final.loc[df_train_final['date_block_num']<32]['item_cnt_day']
X_test= df_train_final.loc[df_train_final['date_block_num']>=32].drop(['item_cnt_day'],axis=1)
Y_test= df_train_final.loc[df_train_final['date_block_num']>=32]['item_cnt_day']

In [96]:
merged.drop(['date_block_num'],inplace=True,axis=1)


## Model Training

In [46]:
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score
# from xgboost import XGBRegressor 
# cv = KFold(n_splits=10, random_state=1, shuffle=True)

In [ ]:
# X=df_train_final.drop(['item_cnt_day'],axis=1)
# Y=Y_train=df_train_final['item_cnt_day']

In [ ]:
# xg = XGBRegressor(learning_rate=0.5, n_estimators=50, max_depth=4, random_state=4)
# kfold = KFold(n_splits=10, random_state=7)
              
# scores = cross_val_score(model, X, Y, scoring='neg_root_mean_squared_error', cv=cv, n_jobs=-1)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
# results_y = np.abs(scores.reshape(-1,1))
# print(results_y)

[[1.90243256]
 [2.34071601]
 [4.32159841]
 [2.28881376]
 [2.7777308 ]
 [2.70885927]
 [2.04119317]
 [1.87175292]
 [1.97970571]
 [2.05793618]]


In [97]:
param = {"objective":"reg:squarederror"} 
dtrain = xgb.DMatrix(X_train, Y_train, missing=-99)
dvalid = xgb.DMatrix(X_test, Y_test, missing=-99)

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
xgb_r = xgb.train(params = param, dtrain = dtrain, evals=watchlist, num_boost_round = 200,early_stopping_rounds=20, verbose_eval=True) 
#pred = np.round(xgb_r.predict(test_dmatrix) )

[0]	train-rmse:1.23389	eval-rmse:2.25813
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[1]	train-rmse:1.07745	eval-rmse:1.71842
[2]	train-rmse:1.05178	eval-rmse:1.70227
[3]	train-rmse:0.953069	eval-rmse:1.3255
[4]	train-rmse:0.942082	eval-rmse:1.31784
[5]	train-rmse:0.931218	eval-rmse:1.28588
[6]	train-rmse:0.927441	eval-rmse:1.27841
[7]	train-rmse:0.871141	eval-rmse:1.04806
[8]	train-rmse:0.839774	eval-rmse:1.04766
[9]	train-rmse:0.834648	eval-rmse:1.04204
[10]	train-rmse:0.831215	eval-rmse:1.0419
[11]	train-rmse:0.828644	eval-rmse:1.03465
[12]	train-rmse:0.825069	eval-rmse:1.03278
[13]	train-rmse:0.823172	eval-rmse:1.03238
[14]	train-rmse:0.788738	eval-rmse:1.03278
[15]	train-rmse:0.766418	eval-rmse:0.940029
[16]	train-rmse:0.764301	eval-rmse:0.940538
[17]	train-rmse:0.760444	eval-rmse:0.948953
[18]	train-rmse:0.75779	eval-rmse:0.949205
[19]	train-rmse:0.737027	eval-rmse:0.952879
[20]	tra

In [98]:
pred = np.round(xgb_r.predict(dvalid) )

In [99]:
X_test['predictions']=pred
X_test['item_cnt_day']=Y_test

In [100]:
X_test['predictions'].value_counts().head(2)

1.0    59467
2.0     1310
Name: predictions, dtype: int64

In [106]:
param = {"objective":"reg:squarederror"} 
#df_train_final.sample(frac=1)
X_train=df_train_final.drop(['item_cnt_day'],axis=1)
Y_train=df_train_final['item_cnt_day']

train_dmatrix = xg.DMatrix(data = X_train, label = Y_train) 
watchlist = [(train_dmatrix, 'train')]
xgb_r = xg.train(params = param, dtrain = train_dmatrix, evals=watchlist, early_stopping_rounds=5, num_boost_round = 100) 

[0]	train-rmse:1.28751
Will train until train-rmse hasn't improved in 5 rounds.
[1]	train-rmse:1.1074
[2]	train-rmse:1.08351
[3]	train-rmse:0.97832
[4]	train-rmse:0.962676
[5]	train-rmse:0.951442
[6]	train-rmse:0.94315
[7]	train-rmse:0.936831
[8]	train-rmse:0.875023
[9]	train-rmse:0.837809
[10]	train-rmse:0.801815
[11]	train-rmse:0.775052
[12]	train-rmse:0.769702
[13]	train-rmse:0.766266
[14]	train-rmse:0.746434
[15]	train-rmse:0.74397
[16]	train-rmse:0.729067
[17]	train-rmse:0.727551
[18]	train-rmse:0.725282
[19]	train-rmse:0.723178
[20]	train-rmse:0.721575
[21]	train-rmse:0.721299
[22]	train-rmse:0.720013
[23]	train-rmse:0.708415
[24]	train-rmse:0.707374
[25]	train-rmse:0.706805
[26]	train-rmse:0.705501
[27]	train-rmse:0.705175
[28]	train-rmse:0.704975
[29]	train-rmse:0.703751
[30]	train-rmse:0.702962
[31]	train-rmse:0.702572
[32]	train-rmse:0.701987
[33]	train-rmse:0.700725
[34]	train-rmse:0.70029
[35]	train-rmse:0.700026
[36]	train-rmse:0.699695
[37]	train-rmse:0.698301
[38]	train-

In [107]:
test_dmatrix=xg.DMatrix(data = df_test_final.drop(['ID','item_cnt_day'],axis=1)) 
pred = np.round(xgb_r.predict(test_dmatrix))

In [108]:
submission=pd.DataFrame()
submission['ID']=df_test_final['ID'].astype(int)
submission['item_cnt_month']=pred

In [109]:
submission['item_cnt_month'].value_counts().head(2)

1.0    200450
2.0      7882
Name: item_cnt_month, dtype: int64

In [110]:
submission=submission.sort_values('ID')
submission.head(2)

,ID,item_cnt_month
0,0,1.0
42,1,1.0


In [112]:
submission.to_csv('Round 6.1 | XGB | Item Cat | Sales_Mean.csv',index=False)
